In [ ]:
! pip install numpy pandas pgvector langchain transformers google-cloud-aiplatform psycopg2-binary shapely==2.0.6 langchain-core langchain-community langchain-google-vertexai 

In [2]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

## Download and load the dataset

In [1]:
import os
import pandas as pd
import vertexai
from vertexai.language_models import TextEmbeddingModel
from vertexai.generative_models import GenerativeModel
from IPython.display import display, Markdown

from langchain_google_vertexai import VertexAIEmbeddings
import vertexai

PROJECT_ID = "qwiklabs-gcp-02-3482f55918a6"  # @param {type:"string"}
REGION = "us-central1"  # @param {type:"string"}

# Initialize Vertex AI SDK
vertexai.init(project=PROJECT_ID, location=REGION)

Read the CSV dataset into a pandas DataFrame, and saves the DataFrame to a table named products in the AlloyDB cluster.

In [2]:
import psycopg2 # library for PostgreSQL

# Replace with your AlloyDB cluster credentials
cluster_ip_address = "10.102.0.2"
database_user = "postgres"
database_password = "postgres"

# Set environment variables for psql connection
os.environ["PGHOST"] = cluster_ip_address
os.environ["PGUSER"] = database_user
os.environ["PGPASSWORD"] = database_password

# Establish a connection to the database
try:
    conn = psycopg2.connect(
        host=cluster_ip_address,
        user=database_user,
        password=database_password
    )
    print("Connected to the database successfully!")
except Exception as e:
    print("Connection error:", e)
exit(1)

# Read the dataset from the URL
DATASET_URL = "https://github.com/GoogleCloudPlatform/python-docs-samples/raw/main/cloud-sql/postgres/pgvector/data/retail_toy_dataset.csv"
df = pd.read_csv(DATASET_URL)

# Select desired columns and drop missing values
df = df.loc[:, ["product_id", "product_name", "description", "list_price"]]
df = df.dropna()

# Save the DataFrame to the AlloyDB cluster
df.to_sql('products', con=f'postgresql://{cluster_ip_address}', if_exists='replace', index=False)

# Retrieve data from the 'products' table
cur = conn.cursor()
cur.execute("SELECT * FROM products")
results = cur.fetchall()

# Close the connection
conn.close()
print(results[5])

Connected to the database successfully!
('74a695e3675efc2aad11ed73c46db29b', 'Slip N Slide Triple Racer with Slide Boogies', 'Triple Racer Slip and Slide with Boogie Boards. The unit is 16 foot long. The unit has 3 sliding lanes.', 37.21)


## Generate Vector Embeddings using a Text Embedding Model

Preprocess product descriptions, generate vector embeddings for them, and store the embeddings along with other relevant data in a PostgreSQL database table for downstream analysis or applications.

We import the RecursiveTextSplitter class from the LangChain library, which is used for splitting text into smaller chunks. Iterate through each row in the DataFrame df and extract the product ID and description from each row.

Then, we will split each description into smaller chunks and will create a dictionary for each chunk.

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    separators=[".", "\n"],
    chunk_size=500,
    chunk_overlap=0,
    length_function=len,
)
max_documents = 60
chunked = []
for index, row in df.iterrows():
    product_id = row["product_id"]
    desc = row["description"]
    splits = text_splitter.create_documents([desc])
    if len(chunked) < max_documents:
        for s in splits:
            r = {"product_id": product_id, "content": s.page_content}
            chunked.append(r)
    else:
        break
print(len(chunked))

61


Process product descriptions from a dataset by splitting them into smaller chunks, sending them to Vertex AI for embedding generation, and storing the retrieved embeddings back into the data structure.

In [4]:
# Generate the vector embeddings for each chunk of text.
# This code snippet may run for a few minutes.
from langchain_google_vertexai import VertexAIEmbeddings
from langchain_google_vertexai import VertexAI
from langchain_google_vertexai import VertexAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from google.cloud import aiplatform
import time

embeddings = VertexAIEmbeddings("text-embedding-004")

# Helper function to retry failed API requests with exponential backoff.
def retry_with_backoff(func, *args, retry_delay=5, backoff_factor=2, **kwargs):
    max_attempts = 10
    retries = 0
    for i in range(max_attempts):
        try:
            return func(*args, **kwargs)
        except Exception as e:
            print(f"error: {e}")
            retries += 1
            wait = retry_delay * (backoff_factor**retries)
            print(f"Retry after waiting for {wait} seconds...")
            time.sleep(wait)

batch_size = 5
for i in range(0, len(chunked), batch_size):
    request = [x["content"] for x in chunked[i : i + batch_size]]
    response = retry_with_backoff(embeddings.embed_documents, request)
    # Store the retrieved vector embeddings for each chunk back.
    for x, e in zip(chunked[i : i + batch_size], response):
        x["embedding"] = e

# Store the generated embeddings in a pandas dataframe.
product_embeddings = pd.DataFrame(chunked)
product_embeddings.head()

,product_id,content,embedding
0,7e8697b5b7cdb5a40daf54caf1435cd5,"Rock, paper, scissors is a great way to resolv...","[-0.012742510065436363, -0.017762525007128716,..."
1,7e8697b5b7cdb5a40daf54caf1435cd5,". Great for educational games, dice games, boa...","[-0.02675631456077099, -0.024180172011256218, ..."
2,7de8b315b3cb91f3680eb5b88a20dcee,Turn any small bicycle into an instrument for ...,"[-0.035151511430740356, -0.06757935136556625, ..."
3,7de8b315b3cb91f3680eb5b88a20dcee,. Durable Construction: Steel brackets stand u...,"[-0.043607085943222046, -0.03579239919781685, ..."
4,7de8b315b3cb91f3680eb5b88a20dcee,. Tools required: Adjustable wrench. www.schwi...,"[-0.03578212484717369, -0.04733877256512642, 0..."


Create a table called product_embeddings with columns for product IDs, text content, and vector embeddings. We will iterate through the product embeddings generated previously and insert or update them in the table.


In [5]:
import psycopg2
import numpy as np
from pgvector.psycopg2 import register_vector

# AlloyDB cluster connection details (replace with your actual values)
cluster_ip_address = "10.102.0.2"
database_user = "postgres"
database_password = "postgres"

# Connect to AlloyDB cluster
conn = psycopg2.connect(
    host=cluster_ip_address,
    user=database_user,
    password=database_password
)

# Create cursor for executing SQL commands
cur = conn.cursor()

# Ensure vector extension is installed
cur.execute("CREATE EXTENSION IF NOT EXISTS vector")

# Drop existing table (if it exists)
cur.execute("DROP TABLE IF EXISTS product_embeddings")

# Create the `product_embeddings` table
cur.execute("""
    CREATE TABLE product_embeddings(
        product_id VARCHAR(1024) NOT NULL PRIMARY KEY,
        content TEXT,
        embedding vector(768)
    )
""")

# Register the vector type
register_vector(conn)

# Store vector embeddings in the table
for index, row in product_embeddings.iterrows():
    cur.execute("SELECT EXISTS(SELECT 1 FROM product_embeddings WHERE product_id = %s)", (row["product_id"],))
    if not cur.fetchone()[0]:  # Product ID doesn't exist, insert
        cur.execute("INSERT INTO product_embeddings (product_id, content, embedding) VALUES (%s, %s, %s)", (row["product_id"], row["content"], row["embedding"]))
    else:  # Product ID exists, update
        cur.execute("UPDATE product_embeddings SET content = %s, embedding = %s WHERE product_id = %s", (row["content"], row["embedding"], row["product_id"]))


# Commit changes and close connection
conn.commit()
conn.close()
print("Created the 'product_embeddings' table successfully")

Created the 'product_embeddings' table successfully


## Create Indexes for faster Similarity Search

Vector indexes can significantly speed up similarity search operations and avoid the brute-force exact nearest neighbor search that is used by default.

Pgvector comes with two types of indexes: hnsw and ivfflat.

We build an HNSW index on product_embeddings table using cosine similarity metric for faster search based on descriptions.

In [6]:
import psycopg2
from pgvector.psycopg2 import register_vector

# AlloyDB cluster connection details (replace with your actual values)
cluster_ip_address = "10.102.0.2"
database_user = "postgres"
database_password = "postgres"

m = 24
ef_construction = 100
operator = "vector_cosine_ops"

# Connect to AlloyDB cluster
conn = psycopg2.connect(
    host=cluster_ip_address,
    user=database_user,
    password=database_password
)

# Register the vector type
register_vector(conn)

# Create the HNSW index on the `product_embeddings` table
cur = conn.cursor()
cur.execute(
    f"""CREATE INDEX ON product_embeddings
        USING hnsw(embedding {operator})
        WITH (m = {m}, ef_construction = {ef_construction})
    """
)
conn.commit()

# Close the connection
conn.close()
print("Created an HNSW Index successfully")


Created an HNSW Index successfully


Next, create an IVFFLAT index on the product_embeddings table using cosine similarity for swift similarity searches among product descriptions.

In [7]:
import psycopg2
from pgvector.psycopg2 import register_vector

# AlloyDB cluster connection details (replace with your actual values)
cluster_ip_address = "10.102.0.2"
database_user = "postgres"
database_password = "postgres"

lists = 100
operator = "vector_cosine_ops"

# Connect to AlloyDB cluster
conn = psycopg2.connect(
    host=cluster_ip_address,
    user=database_user,
    password=database_password
)

# Register the vector type
register_vector(conn)

# Create the IVFFLAT index on the `product_embeddings` table
cur = conn.cursor()
cur.execute(
    f"""CREATE INDEX ON product_embeddings
        USING ivfflat(embedding {operator})
        WITH (lists = {lists})
    """
)
conn.commit()

# Close the connection
conn.close()
print("Created an IVFFLAT Index successfully")


Created an IVFFLAT Index successfully


Now, we will conduct the similarity search. The provided code identifies products most relevant to the user's query based on their textual descriptions. To ensure relevant results, the code also filters based on the desired price range 25-100.

In [8]:
import psycopg2
from pgvector.psycopg2 import register_vector
import pandas as pd

# AlloyDB cluster connection details (replace with your actual values)
cluster_ip_address = "10.102.0.2"
database_user = "postgres"
database_password = "postgres"

toy = "playing card games"
min_price = 25
max_price = 100

# Connect to AlloyDB cluster
conn = psycopg2.connect(
    host=cluster_ip_address,
    user=database_user,
    password=database_password
)

# Register the vector type
register_vector(conn)

# Get the query embedding
qe = embeddings.embed_query("toy")

# Perform the similarity search and filtering
cur = conn.cursor()
similarity_threshold = 0.1
num_matches = 50
# Pass 'qe' twice to match the number of placeholders in the query
cur.execute(
    """
    WITH vector_matches AS (
        SELECT product_id, 1 - (embedding <=> %s::vector) AS similarity
        FROM product_embeddings
        WHERE 1 - (embedding <=> %s::vector) > %s
        ORDER BY similarity DESC
        LIMIT %s
    )
    SELECT product_name, list_price, description
    FROM products
    WHERE product_id IN (SELECT product_id FROM vector_matches)
    AND list_price >= %s AND list_price <= %s
    """,
    (qe, qe, similarity_threshold, num_matches, min_price, max_price)
)
results = cur.fetchall()

# Process the results
matches = []
for r in results:
    try:
        list_price = round(float(r[2]), 2)  # Attempt conversion and rounding
    except ValueError:
        list_price = r[2] 
    matches.append({
        "product_name": r[0],
        "list_price": r[1],
        "description": r[2]
    })

# Display the results
matches_df = pd.DataFrame(matches)
print(matches_df.head(5))

# Close the connection
conn.close()


                                        product_name  list_price  \
0                    12"-20" Schwinn Training Wheels       28.17   
1       Slip N Slide Triple Racer with Slide Boogies       37.21   
2  Polaris 39-310 5-Liter Zippered Super Bag for ...       39.47   
3  Sandbox Castle 2-in-1 Sand and Water Table wit...       60.49   
4  Jensen S100T Commercial Tot Full Bucket Rubber...       90.18   

                                         description  
0  Turn any small bicycle into an instrument for ...  
1  Triple Racer Slip and Slide with Boogie Boards...  
2  Keep your pool water sparkling clean all seaso...  
3  Package Includes Sandbox Castle 2-in-1 Sand an...  
4  This is a fully enclosed one piece infant seat...  


## LLMs and LangChain

### Use case 1: Building an AI-curated contextual hybrid search

Combine natural language query text with regular relational filters to create a powerful hybrid search.

In [10]:
user_query = "Do you have a toy set that teaches numbers and letters to kids?"  # @param {type:"string"}
min_price = 20  # @param {type:"integer"}
max_price = 100  # @param {type:"integer"}

Generate the vector embedding for the user query.

In [11]:
qe = embeddings.embed_query(user_query)

Use pgvector to find similar products. The pgvector similarity search operators provide powerful semantics to combine the vector search operation with regular query filters in a single SQL query.

In [12]:
import psycopg2
from psycopg2 import sql
from pgvector.psycopg2 import register_vector

def main(user_query,min_price,max_price):
    try:
        # AlloyDB cluster connection details (replace with your actual values)
        cluster_ip_address = "10.102.0.2"
        database_user = "postgres"
        database_password = "postgres"

        # Connect to AlloyDB cluster
        conn = psycopg2.connect(
        host=cluster_ip_address,
        user=database_user,
        password=database_password)

        # Register the vector type
        register_vector(conn)

        # Get the query embedding
        qe = embeddings.embed_query(user_query)

        # Perform the similarity search and filtering
        cur = conn.cursor()
        similarity_threshold = 0.1
        num_matches = 50

        # Pass 'qe' twice to match the number of placeholders in the query
        cur.execute(
                    """
                        WITH vector_matches AS (
                        SELECT product_id, 1 - (embedding <=> %s::vector) AS similarity
                        FROM product_embeddings
                        WHERE 1 - (embedding <=> %s::vector) > %s
                        ORDER BY similarity DESC
                        LIMIT %s
                )
                SELECT product_name, list_price, description
                FROM products
                WHERE product_id IN (SELECT product_id FROM vector_matches)
                AND list_price >= %s AND list_price <= %s
                """,
                (qe, qe, similarity_threshold, num_matches, min_price, max_price))
        results = cur.fetchall()
        # Process the results
        matches = []
        for r in results:
            try:
                list_price = round(float(r[2]), 2)  # Attempt conversion and rounding
            except ValueError:
                    list_price = r[2] 
                    matches.append({
                        "product_name": r[0],
                        "list_price": r[1],
                        "description": r[2]
                    })
                    # Display the results
        matches_df = pd.DataFrame(matches)
        print(matches_df.head(5))


    except Exception as e:
        print(f"Error during database operations: {e}")
    finally:
        # Close the connection
        conn.close()
    return

# Call the main function (no need for asyncio in this context)
main("Do you have a toy set that teaches numbers and letters to kids?",25,100)

                                        product_name  list_price  \
0                    12"-20" Schwinn Training Wheels       28.17   
1       Slip N Slide Triple Racer with Slide Boogies       37.21   
2  Polaris 39-310 5-Liter Zippered Super Bag for ...       39.47   
3  Sandbox Castle 2-in-1 Sand and Water Table wit...       60.49   
4  Jensen S100T Commercial Tot Full Bucket Rubber...       90.18   

                                         description  
0  Turn any small bicycle into an instrument for ...  
1  Triple Racer Slip and Slide with Boogie Boards...  
2  Keep your pool water sparkling clean all seaso...  
3  Package Includes Sandbox Castle 2-in-1 Sand an...  
4  This is a fully enclosed one piece infant seat...  


Use LangChain to summarize and generate a high-quality prompt to answer the user query.

After finding the similar products and their descriptions using pgvector, the next step is to use them for generating a prompt input for the LLM model. Since individual product descriptions can be very long, they may not fit within the specified input payload limit for an LLM model. The MapReduceChain from the LangChain framework is used to generate and combine short summaries of similarly matched products. The combined summaries are then used to build a high-quality prompt for an input to the LLM model.

In [13]:
# Using LangChain for summarization and efficient context building.
from IPython.display import display, Markdown

import langchain_core
from langchain.chains.summarize import load_summarize_chain
from langchain_core.documents.base import Document
from langchain_google_vertexai import VertexAI
from langchain_google_vertexai import VertexAIEmbeddings
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate

llm = VertexAI(model_name="gemini-1.5-flash")

map_prompt_template = """
            You will be given a detailed description of a toy product.
            This description is enclosed in triple backticks (```).
            Using this description only, extract the name of the toy,
            the price of the toy and its features.

            ```{text}```
            SUMMARY:
            """
map_prompt = PromptTemplate(template=map_prompt_template, input_variables=["text"])

combine_prompt_template = """
                You will be given a detailed description of different toy products
                enclosed in triple backticks (```) and a question enclosed in
                double backticks(``).
                Select one toy that is most relevant to answer the question.
                Using that selected toy description, answer the following
                question in as much detail as possible.
                You should only use the information in the description.
                Your answer should include the name of the toy, the price of the toy
                and its features. Your answer should be less than 200 words.
                Your answer should be in Markdown in a numbered list format.

                Description:
                ```{text}```

                Question:
                ``{user_query}``

                Answer:
                """
combine_prompt = PromptTemplate(
    template=combine_prompt_template, input_variables=["text", "user_query"]
)

docs = [Document(page_content=str(t)) for t in matches]
chain = load_summarize_chain(
    llm, chain_type="map_reduce", map_prompt=map_prompt, combine_prompt=combine_prompt
)

# Invoke the chain
output = chain.invoke({
    "input_documents": docs,
    "user_query": user_query,
})

# Extract the output_text
answer = output.get('output_text', ' ')

# Display the answer in Markdown format
display(Markdown(answer))

The toy that is most relevant to the question is the **Sandbox Castle 2-in-1 Sand and Water Table with Beach Playset**. 

Here are the details of the toy:

1. **Toy Name:** Sandbox Castle 2-in-1 Sand and Water Table with Beach Playset
2. **Price:** $60.49
3. **Features:**
    - Includes 14-piece sand tools including watering can, shovel, rake, castle and shell sand molds, sailboat, and bridges. These tools can be used for imaginative play and to teach children about different shapes, sizes, and colors.
    - This toy is an activity for indoor or outdoor play, making it a great option for learning and fun.
    - The toy is small enough to take on trips, making it a portable option for learning. 


### Use case 2: Adding AI-powered creative content generation

Use knowledge from the existing dataset to generate new AI-powered content from an initial prompt.

A third-party seller on the retail platform wants to use the AI-powered content generation to create a detailed description of their new bicycle product.



In [15]:
# Please fill in these values.
creative_prompt = "A bicycle with brand name 'Roadstar bike' for kids that comes with training wheels and helmet." 

Leverage the pgvector similarity search operator to find an existing product description that closely matches the new product specified in the initial prompt.

In [16]:
import psycopg2
from pgvector.psycopg2 import register_vector

def main():
    try:
        # AlloyDB cluster connection details
        cluster_ip_address = "10.102.0.2"
        database_user = "postgres"
        database_password = "postgres"

        # Connect to AlloyDB cluster
        conn = psycopg2.connect(
            host=cluster_ip_address,
            user=database_user,
            password=database_password
        )

        # Register the vector type
        register_vector(conn)

        # Get the query embedding
        qe = embeddings.embed_query(creative_prompt)

        # Print the query embedding for debugging

        # Ensure qe is a non-empty vector
        if not qe:
            print("Error: The query embedding is empty.")
            return

        # Create the embedding string
        matches = []
        similarity_threshold = 0.5 

        # Perform the similarity search and filtering
        cur = conn.cursor()

        cur.execute(
            """
            WITH vector_matches AS (
                SELECT product_id, 1 - (embedding <=> %s::vector) AS similarity
                FROM product_embeddings
                WHERE 1 - (embedding <=> %s::vector) > %s
                ORDER BY similarity DESC
                LIMIT 1
            )
            SELECT description FROM products
            WHERE product_id IN (SELECT product_id FROM vector_matches)
            """,
            (qe, qe, similarity_threshold)
        )

        results = cur.fetchall()

        # Process the results
        for r in results:
            matches.append(r[0])

        if not matches:
            print("No matches found.")
        else:
            print(matches)

    except Exception as e:
        print(f"Error during database operations: {e}")
    finally:
        # Close the connection if it was established
        if conn:
            conn.close()

# Call the main function
main()

['Turn any small bicycle into an instrument for learning to ride with the Schwinn 12"-20" Training Wheels. They feature a slotted design to fit 12" to 20" bikes. The training wheels are easy to assemble, install and remove, so that when your little one is able to ride without assistance, you can take them off. These bicycle training wheels include steel brackets and rubber tires that can stand up to heavy use. Training Wheels, Fits 12 inches - 20 inches bicycles. Est. 1895. Durable Construction: Steel brackets stand up to heavy use. Customizable: Two sets of wheel decals included. Features: Fits Most Childrens Bicycles: Intended for 12 inch - 20 inch bicycles. Steel Brackets: Offer increased durability. Includes two sets of wheel decals: Learn how to ride in style - see images below. Easy to Adjust: Slotted design for size adjustment. Includes: One pair of training wheels, four decals, installation instructions, and all mounting hardware. Tools required: Adjustable wrench. www.schwinnb

Use the existing matched product description as the prompt context to generate new creative output from the LLM.

In [17]:
from IPython.display import display, Markdown
from langchain_google_vertexai import VertexAI
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableSequence

template = """
            You are given descriptions about some similar kind of toys in the context.
            This context is enclosed in triple backticks (```).
            Combine these descriptions and adapt them to match the specifications in
            the initial prompt. All the information from the initial prompt must
            be included. You are allowed to be as creative as possible,
            and describe the new toy in as much detail. Your answer should be
            in markdown in lists and less than 200 words.

            Context:
            ```{context}```

            Initial Prompt:
            {creative_prompt}

            Answer:
        """

prompt = PromptTemplate(
    template=template, input_variables=["context", "creative_prompt"]
)

# Increase the `temperature` to allow more creative writing freedom.
llm = VertexAI(model_name="gemini-1.5-flash", temperature=0.7)

# Assuming each dictionary in `matches` has a `description` key:
context = "\n".join(
    match["description"] for match in matches if isinstance(match, dict)
)

# Use RunnableSequence instead of LLMChain
llm_chain = RunnableSequence(prompt | llm)

# Invoke the chain
answer = llm_chain.invoke({
    "context": context,
    "creative_prompt": creative_prompt,
})

# Display the answer in Markdown format
display(Markdown(answer))

Here is a description of the Roadstar Bike for kids: 

* **Roadstar Bike:** This stylish and durable bike is perfect for young riders learning to ride. It features a bright, colorful design with a sturdy frame that can handle the bumps and scrapes of childhood adventures. 
* **Training Wheels:**  The Roadstar Bike comes equipped with adjustable training wheels.  These wheels are easy to install and remove, allowing you to customize the bike as your child progresses.  
* **Helmet:** Safety first!  The Roadstar Bike includes a matching helmet with fun graphics.  The helmet is adjustable for a snug and comfortable fit. 
* **Additional Features:** 
    * **Handlebar Grips:** Soft, comfortable grips for little hands.
    * **Bell:**  Let everyone know you're coming!
    * **Kickstand:** Easy to park and store. 
